In [3]:
import requests
import urllib.parse
import hashlib
import hmac
import base64
import time
from datetime import datetime as dt, timedelta as td

# Other necessary imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.io as pio
import json
import pandas as pd
#import altair as alt
import numpy as np
import os
import config as cfg

from difflib import get_close_matches

In [81]:
api_url = "https://api.kraken.com"

api_endpoints = {
    'Assets': '/0/public/Assets',
    'AssetPairs': '/0/public/AssetPairs',
    'Ticker': '/0/public/Ticker',
    'OHLC': '/0/public/OHLC',
    'default OHLC': '/0/public/OHLC',

    'Balance': '/0/private/Balance',
    'ExtendedBalance': '/0/private/BalanceEx',
    'TradeBalance': '/0/private/TradeBalance',
    'Ledgers': '/0/private/Ledgers',
    'QueryLedgers': '/0/private/QueryLedgers',
    'TradeVolume': '/0/private/TradeVolume',
    'TradesHistory': '/0/private/TradesHistory',
    'OpenOrders': '/0/private/OpenOrders',

}

# Read Kraken API key and secret stored in config file
api_key = cfg.api_key
api_sec = cfg.api_priv


In [6]:
# Function to generate a nonce
def generate_nonce():
    nonce = str(int(1000 * time.time()))
    return nonce

In [7]:
# Function to get Kraken signature
def get_kraken_signature(urlpath, data, secret):
    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()

    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode() 

In [8]:
# Function to make a POST Kraken API request
def kraken_request(uri_path, data, api_key, api_sec, headers=None):
    if headers is None:
        headers = {}

    headers['API-Key'] = api_key
    headers['API-Sign'] = get_kraken_signature(uri_path, data, api_sec)
    req = requests.post((api_url + uri_path), headers=headers, data=data)
    return req

# Function to make a GET request to the Kraken API
def kraken_get_request(uri_path, data=None, headers=None):
    # Default headers
    if headers is None:
        headers = {'Accept': 'application/json'}

    # Construct the request URL
    if uri_path == api_endpoints['OHLC']:
        # Append query parameters for the OHLC endpoint
        query_params = f"?pair={data['pair']}&interval={data['interval']}"
        full_url = api_url + uri_path + query_params
    else:
        # Use uri_path directly for other endpoints
        full_url = api_url + uri_path

    # Make the GET request
    req = requests.get(full_url, headers=headers, data=data)
    return req

In [9]:
# Function to grab the current GBPUSD rate from the Kraken API
def grab_rate():
    resp = kraken_get_request(api_endpoints['Ticker'], {"pair": 'GBPUSD'}).json()
    rate = 1/float(resp['result']['ZGBPZUSD']['c'][0])
    return rate

In [10]:
grab_rate()

0.7954753362871985

In [63]:
# Function to grab all tradeable asset pairs on Kraken, filtering out pairs quoted in both USD and GBP, preference for USD pairs
def grab_all_assets():
    # Construct the Kraken API request and get all asset pairs from the Kraken API
    assetPairs = kraken_get_request(api_endpoints['AssetPairs']).json()['result']
    
    # Filter out asset pairs that are not quoted in USD
    assetPairs = {k: v for k, v in assetPairs.items() if v['quote'] == 'ZUSD' or v['quote'] == 'ZGBP'}

    # Remove gbp usd duplicates, keeping usd pairs - usually higher volumes
    for asset in list(assetPairs.keys()):
        if assetPairs[asset]['altname'].endswith('GBP') and assetPairs[asset]['altname'][:-3] + 'USD' in assetPairs:
            assetPairs.pop(asset)

    # Extract 'altname' for each asset pair
    altNames = [details['altname'] for details in assetPairs.values()]
    return altNames, assetPairs# altNames is a list of all tradeable asset pairs on Kraken example: ['XXBTZUSD', 'XETHZUSD', 'XETHXXBT']

In [66]:
assets, assetPairs = grab_all_assets()
assets

# Printing non duplicate GBP pairs
for asset in assetPairs:
    if assetPairs[asset]['altname'][-3:] == 'GBP':
        if assetPairs[asset]['altname'][:-3] + 'USD' not in assetPairs.keys():
            print(asset)

EURGBP
LTCGBP
USDTGBP
XETHZGBP
XRPGBP
XXBTZGBP
XXLMZGBP


In [73]:
def grab_ext_bal():
    # Construct the Kraken API request and get the External Balance Information
    resp = kraken_request(api_endpoints['ExtendedBalance'], {"nonce": generate_nonce(),}, api_key, api_sec).json()
    balanceDict = {}
    # Extract the balance of each asset and the asset name
    # Use a dictionary comprehension to build the balanceDict
    balanceDict = {
        asset: float(details['balance'])
        for asset, details in resp['result'].items()
        if float(details['balance']) != 0
    }
    return balanceDict # balanceDict is a dictionary with asset names as keys and balances as values example: {'XBT': 0.1, 'GBP': 1000}


In [74]:
grab_ext_bal()

{'SOL.F': 0.1253581071,
 'XXRP': 11.37776632,
 'ZGBP': 263.9041,
 'ZUSD': 107.6286}

In [180]:
def grab_clean_bal():
    balanceDict = grab_ext_bal()
    cashBalanceDict = {asset: balance for asset, balance in balanceDict.items() if asset.startswith('Z')}
    
    balanceDictPairs = {
        match[0]: balance
        for asset, balance in balanceDict.items()
        if not asset.startswith('Z') and (match := get_close_matches(asset + "USD", assets, n=1))
    }
    
    return balanceDict, balanceDictPairs, cashBalanceDict

In [181]:
cleanBal = grab_clean_bal()
cleanBal

({'SOL.F': 0.1249334871,
  'XXRP': 11.37776632,
  'ZGBP': 263.9041,
  'ZUSD': 107.5484},
 {'SOLUSD': 0.1249334871, 'XRPUSD': 11.37776632},
 {'ZGBP': 263.9041, 'ZUSD': 107.5484})

In [182]:
def grab_trade_bal():
    # Construct the Kraken API request and get the Trade Balance Information
    resp = kraken_request(api_endpoints['TradeBalance'], {"nonce": generate_nonce(), "asset": 'ZUSD'}, api_key, api_sec).json()
    
    tradeBalanceDict = resp['result']

    # Mapping keys to their meanings from API documentation
    key_mapping = {
        'eb': 'Extended Balance',
        'tb': 'Trade Balance',
        'm': 'Margin Amount',
        'n': 'Unrealized Net Profit/Loss',
        'c': 'Cost Basis',
        'v': 'Current Floating Valuation',
        'e': 'Equity',
        'mf': 'Free Margin',
        'ml': 'Margin Level',
        'uv': 'Unexecuted Value',
    }
    tradeBalanceDict.update({key_mapping[k]: tradeBalanceDict.pop(k) for k in key_mapping if k in tradeBalanceDict})

    return tradeBalanceDict # tradeBalanceDict is a dictionary with trade balance information


In [183]:
grab_trade_bal()

{'Extended Balance': '489.1797',
 'Trade Balance': '484.3337',
 'Margin Amount': '409.9447',
 'Unrealized Net Profit/Loss': '-16.2053',
 'Cost Basis': '1579.8174',
 'Current Floating Valuation': '1596.0226',
 'Equity': '468.1284',
 'Free Margin': '58.1837',
 'Margin Level': '114.19',
 'Unexecuted Value': '0.0000'}

In [188]:
def grab_ticker_data(assetPairs):
    # example assetPairs = ['XXBTZUSD', 'XETHZUSD', 'XETHXXBT']
    resp = kraken_get_request(api_endpoints['Ticker']).json()
    
    # Map keys to more descriptive names
    key_mapping = {
        'a': 'Ask',
        'b': 'Bid',
        'c': 'Last Trade Closed',
        'v': 'Volume',
        'p': 'VWAP',
        't': 'Number of Trades',
        'l': 'Low',
        'h': 'High',
        'o': 'Open',
    }
    
    desc = {
        'Ask' : '[Price] [Whole Lot Volume] [Lot Volume]',
        'Bid' : '[Price] [Whole Lot Volume] [Lot Volume]',
        'Last Trade Closed' : '[Price] [Lot Volume]',
        'Volume' : '[Today] [Last 24 Hours]',
        'VWAP' : '[Today] [Last 24 Hours]',
        'Number of Trades' : '[Today] [Last 24 Hours]',
        'Low' : '[Today] [Last 24 Hours]',
        'High' : '[Today] [Last 24 Hours]',
        'Open' : '[Today]',

    }
    # Create the ticker dictionary with remapped keys
    tickerDict = {
        get_close_matches(assetPair, resp['result'].keys(), n=1)[0]: {
            key_mapping[key]: value for key, value in resp['result'][get_close_matches(assetPair, resp['result'].keys(), n=1)[0]].items() if key in key_mapping
        }
        for assetPair in assetPairs
    }
    
    return tickerDict, desc
# tickerDict is a dictionary with asset pairs as keys and ticker data as values example: {'XXBTZUSD': {'a': ['10000.0', '1', '1.000'], 'b': ['9999.0', '1', '1.000'], 'c': ['10000.5', '0.1'], 'v': ['100', '200'], 'p': ['10000.0', '10000.0'], 't': [100, 200], 'l': ['9999.0', '9999.0'], 'h': ['10000.0', '10000.0'], 'o': '10000.0'}}

In [190]:
grab_ticker_data(list(cleanBal[1].keys()))[0]

{'SOLUSD': {'Ask': ['192.81000', '1', '1.000'],
  'Bid': ['192.80000', '88', '88.000'],
  'Last Trade Closed': ['192.80000', '0.29020400'],
  'Volume': ['80959.03219729', '97583.32934549'],
  'VWAP': ['194.32798', '194.22995'],
  'Number of Trades': [13375, 17230],
  'Low': ['191.81000', '191.81000'],
  'High': ['197.19000', '197.19000'],
  'Open': '195.17000'},
 'XRPUSDT': {'Ask': ['2.14311000', '670', '670.000'],
  'Bid': ['2.14310000', '2658', '2658.000'],
  'Last Trade Closed': ['2.14267000', '25.23541400'],
  'Volume': ['395588.96555940', '502291.29910246'],
  'VWAP': ['2.16219367', '2.16804232'],
  'Number of Trades': [1508, 1897],
  'Low': ['2.13934000', '2.13934000'],
  'High': ['2.19643000', '2.20525000'],
  'Open': '2.18505000'}}

In [191]:
# Possible intervals: 1, 5, 15, 30, 60, 240, 1440, 10080, 21600 in minutes i.e., 1 minute, 5 minutes, 15 minutes, 30 minutes, 1 hour, 4 hours, 1 day, 1 week, 1 month
# Possible tenures: 1D (1440), 7D (10080), 1M (43200), 3M (129600), 6M (259200), 1Y (518400) - corresponding intervals are tenure/720 to maximize data points from a single request
possible_intervals =[1, 5, 15, 30, 60, 240, 1440, 10080, 21600]
possible_timeframes = {'1D': 1440, '7D': 10080, '1M': 43200, '3M': 129600, '6M': 259200, '1Y': 518400}

In [193]:
# Function to grab the OHLC data for a given list of asset pairs - todo
def grab_ohlc_data(assetPairs,tenure):
    # divide timerframe by 720 to get the interval but use the next larger closet possible interval
    interval = min([i for i in possible_intervals if i >= possible_timeframes[tenure]/720], default=possible_intervals[-1])
    interval = str(interval)
    # Construct since parameter for the OHLC request using tenure and datetime unix converted timestamp, i.e., subtracting the tenure from the current time and equating it to the since parameter
    since = int(time.time()) - possible_timeframes[tenure]*60
    since = str(since)

    # Construct the Kraken API request and get the OHLC data for the given asset pairs, ohlc grabbing requires use of a temporary endpoint for the OHLC url
    ohlcDict = {}
    for assetPair in assetPairs:
        if assetPair[-3:] == 'USD':
            if assetPair == 'USDTUSD' or assetPair == 'ZGBPZUSD':# or assetPair == 'ETCUSD':    
                continue
            #st.write("X" + assetPair[:3] + "Z") 
            matches = get_close_matches("X" + assetPair[:3] + "Z", list(ohlcDict.keys())[:3], n=1, cutoff = 0.6)
            if matches:#and 'X' not in assetPair:
                continue
            resp = kraken_get_request(api_endpoints['OHLC'], {"pair": assetPair, "interval": interval, "since": since}).json()
            if resp['error'] == KeyError:
                #skip this asset pair
                continue
            ohlcDict[assetPair] = resp['result'][assetPair]
        # To process the response, we need to extract the OHLC data from the response particularly the tick data array and the last timestamp
    # Append the OHLC data to a dataframe and return the dataframe with columns: Time, Open, High, Low, Close, Volume, Count, name it after the asset pair
    return ohlcDict